In [5]:
from mwrogue.esports_client import EsportsClient
site = EsportsClient("lol")


In [14]:
from tqdm.notebook import tqdm
response = (site.cargo_client.query(
    tables="ScoreboardGames=SG, Tournaments=T",
    join_on="SG.OverviewPage=T.OverviewPage",
    fields="T.Name, SG.DateTime_UTC, SG.Team1, SG.Team2"
)

  0%|          | 0/97095 [00:00<?, ?it/s]

In [15]:
print(tqdm(responde))

NameError: name 'responde' is not defined